In [1]:
import sys, time
import numpy as np
import arkouda as ak

In [2]:
ak.connect(server="localhost", port=5555)

4.3.1
psp =  tcp://localhost:5555
connected to tcp://localhost:5555


In [3]:
# files and locations on my(mike's) laptop
hdffiles = ["/Users/valton/lanl_netflow/hdf5/netflow_day-02-00.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-01.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-02.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-03.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-04.hdf",
           "/Users/valton/lanl_netflow/hdf5/netflow_day-02-05.hdf"]

In [4]:
fields = ['srcIP', 'dstIP', 'srcPort', 'dstPort', 'start']

In [5]:
%time nfDF = {field: ak.read_hdf(field, hdffiles) for field in fields}

CPU times: user 1.71 ms, sys: 1.64 ms, total: 3.35 ms
Wall time: 6.22 s


In [6]:
print(nfDF['start'])
print(nfDF['srcIP'])
print(nfDF['dstIP'])
print(nfDF['srcPort'])
print(nfDF['dstPort'])

[118781 118783 118785 ... 140369 140369 140369]
[34111590301 34111676798 48764590996 ... 34112191431 34112191431 34112191431]
[34111773101 34112055194 34111367844 ... 34111545319 31712083968 31712083968]
[5507 3137 5060 ... 78620 44270 49614]
[46272 445 5060 ... 95765 80 80]


In [7]:
print(nfDF)

{'srcIP': array([34111590301, 34111676798, 48764590996, ..., 34112191431, 34112191431, 34112191431]), 'dstIP': array([34111773101, 34112055194, 34111367844, ..., 34111545319, 31712083968, 31712083968]), 'srcPort': array([5507, 3137, 5060, ..., 78620, 44270, 49614]), 'dstPort': array([46272, 445, 5060, ..., 95765, 80, 80]), 'start': array([118781, 118783, 118785, ..., 140369, 140369, 140369])}


In [8]:
print(ak.info(ak.AllSymbols))
print(nfDF['srcIP'].name)

name:"id_5" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_4" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_3" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_2" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8
name:"id_1" dtype:"int64" size:60000000 ndim:1 shape:(60000000) itemsize:8

id_1


In [9]:
%time u,c = ak.unique(nfDF['srcIP'], return_counts=True)

CPU times: user 1.06 ms, sys: 1.25 ms, total: 2.3 ms
Wall time: 15.9 s


In [10]:
print(u.size,u)
print(c.size,c)

25559 [48764522622 48764083067 34111613289 ... 48764332199 34111319955 34111934926]
25559 [132 150 644 ... 175 3084 1497]


In [11]:
%time u,c = ak.unique(nfDF['dstIP'], return_counts=True)

CPU times: user 1.22 ms, sys: 1.64 ms, total: 2.86 ms
Wall time: 16 s


In [12]:
print(u.size,u)
print(c.size,c)

23790 [34111449500 34112062413 34111238959 ... 34112139064 34111598755 34111500102]
23790 [233 14 10214 ... 335 1 39]


In [13]:
%time u,c = ak.unique(nfDF['srcPort'], return_counts=True)

CPU times: user 1.01 ms, sys: 2.1 ms, total: 3.11 ms
Wall time: 1.33 s


In [14]:
print(u.size,u)
print(c.size,c)

64899 [2 4 5 ... 99996 99997 99998]
64899 [188 223 2518 ... 284 220 220]


In [16]:
%time u,c = ak.unique(nfDF['dstPort'], return_counts=True)

CPU times: user 969 µs, sys: 2.25 ms, total: 3.22 ms
Wall time: 1.24 s


In [17]:
print(u.size,u)
print(c.size,c)

59664 [2 4 7 ... 99996 99997 99998]
59664 [4 11 4698 ... 7 5 5]


In [19]:
print((nfDF['srcPort'] == 80).sum())
print((nfDF['dstPort'] == 80).sum())

0
5052897


In [20]:
s = u[0:100] # pick off some dest ports
print(s)
%time t = ak.in1d(nfDF['dstPort'], s) # bool vector of where they are found in the data
print(t.sum())

[2 4 7 ... 119 120 121]
CPU times: user 1.22 ms, sys: 362 µs, total: 1.58 ms
Wall time: 9.65 s
18595194


In [21]:
s = u[0:100].to_ndarray() # pick off some dest ports
dp = nfDF['dstPort'].to_ndarray()
print(s)
%time t = np.in1d(dp, s) # bool vector of where they are found in the data
print(t.sum())

[  2   4   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  24  25  26  27  29  30  31  32  33  34  35  37  38  39  40  41  42
  43  44  45  46  47  48  49  50  51  53  54  56  57  58  60  62  63  64
  65  66  67  68  69  70  72  73  75  77  79  80  81  82  83  85  87  88
  90  92  93  95  97  98  99 100 101 102 104 105 106 107 108 109 110 111
 112 113 114 115 116 117 118 119 120 121]
CPU times: user 5.66 s, sys: 317 ms, total: 5.98 s
Wall time: 6.1 s
18595194


In [22]:
a = nfDF['start']
aMin = a.min()
aMax = a.max()
print(aMin,aMax, aMax - aMin)
%time h = ak.histogram(a,bins=100)
print(h)
print(h.sum(), a.size)

118781 140369 21588
CPU times: user 618 µs, sys: 232 µs, total: 850 µs
Wall time: 134 ms
[87 260 221 ... 507133 811672 769215]
60000000 60000000


In [23]:
a = nfDF['start']
print(a.size, a)
print(ak.is_sorted(a))
%time iv = ak.argsort(a)
print(iv.size, iv)
b = a[iv]
print(b.size, b)

60000000 [118781 118783 118785 ... 140369 140369 140369]
True
CPU times: user 953 µs, sys: 285 µs, total: 1.24 ms
Wall time: 3.39 s
60000000 [0 1 2 ... 59999997 59999998 59999999]
60000000 [118781 118783 118785 ... 140369 140369 140369]


In [ ]:
a = nfDF['dstPort']
print(a.size, a)
print(ak.is_sorted(a))
%time iv = ak.argsort(a)
print(iv.size, iv)
b = a[iv]
print(b.size, b)

In [ ]:
#ak.shutdown()
#ak.disconnect()